In [16]:
import requests
import pandas as pd
from datetime import datetime
import os

In [ ]:
df_lines = pd.read_csv('referentiel-des-lignes.csv', sep=';')
delays_m = pd.read_csv("metro_delays.csv")
delays_r = pd.read_csv("rail_delays.csv")

In [ ]:
line_refs = df_lines[(~df_lines['TransportSubmode'].isin(['suburbanRailway', 'regionalRail', 'railShuttle']))]
line_refs = line_refs[['ID_Line', 'TransportMode', 'Name_Line']]
line_refs = line_refs.sort_values(by='ID_Line')

In [18]:
line_refs

,ID_Line,TransportMode,Name_Line
28,C01371,metro,1
0,C01372,metro,2
38,C01373,metro,3
2,C01374,metro,4
27,C01375,metro,5
35,C01376,metro,6
18,C01377,metro,7
29,C01378,metro,8
41,C01379,metro,9
7,C01380,metro,10


In [19]:
stops_data = pd.read_csv('arrets.csv', sep=';')
stops_data = stops_data[stops_data['ArRType'].isin(['metro', 'rail'])]
stops_data = stops_data.sort_values(by=['ArRType', 'ArRId'])
stops_data = stops_data[['ArRId', 'ArRName', 'ArRType', 'ArRTown']]


In [35]:
stops_filter = [22086, 463013, 22136, 462993, 21964, 462969, 22125, 463113, 41295, 473921, 473993, 41354, 474060, 474061]

filtered_data = stops_data[stops_data['ArRId'].isin(stops_filter)]
filtered_data


,ArRId,ArRName,ArRType,ArRTown
13849,21964,Saint-Lazare,metro,Paris 9e
26086,22086,Charles de Gaulle - Etoile,metro,Paris 17e
13790,22125,Blanche,metro,Paris 9e
32911,22136,Saint-Germain-des-Prés,metro,Paris 6e
34515,462969,Saint-Lazare,metro,Paris 9e
13057,462993,Saint-Germain-des-Prés,metro,Paris 6e
28031,463013,Charles de Gaulle - Etoile,metro,Paris 17e
36557,463113,Blanche,metro,Paris 9e
35241,41295,Avenue du Président Kennedy,rail,Paris 16e
3346,41354,Gare de Buno Gironville,rail,Buno-Bonnevaux


In [46]:
merged_df = pd.merge(delays_m, filtered_data, left_on='stop_reference', right_on='ArRId')
merged_df


,timestamp,stop_reference,stop_name,line_ref,transport_mode,destination_name,departure_status,scheduled_arrival,real_arrival,scheduled_departure,real_departure,arrival_difference,departure_difference,recorded_at_time,ArRId,ArRName,ArRType,ArRTown
0,2024-10-30 01:20:26,22086,Charles de Gaulle-Etoile,STIF:Line::C01371:,metro,Château de Vincennes,delayed,NaN,2024-10-30T00:28:39.675Z,NaN,2024-10-30T00:28:39.675Z,NaN,NaN,2024-10-30T00:19:39.675Z,22086,Charles de Gaulle - Etoile,metro,Paris 17e
1,2024-10-30 02:20:27,22086,Charles de Gaulle-Etoile,STIF:Line::C01371:,metro,Château de Vincennes,delayed,NaN,2024-10-30T01:28:52.695Z,NaN,2024-10-30T01:28:52.695Z,NaN,NaN,2024-10-30T01:19:52.695Z,22086,Charles de Gaulle - Etoile,metro,Paris 17e
2,2024-10-30 03:20:24,22086,Charles de Gaulle-Etoile,STIF:Line::C01371:,metro,Château de Vincennes,delayed,NaN,2024-10-30T02:26:05.147Z,NaN,2024-10-30T02:26:05.147Z,NaN,NaN,2024-10-30T02:17:05.147Z,22086,Charles de Gaulle - Etoile,metro,Paris 17e
3,2024-10-30 04:20:31,22086,Charles de Gaulle-Etoile,STIF:Line::C01371:,metro,Château de Vincennes,delayed,NaN,2024-10-30T03:29:28.250Z,NaN,2024-10-30T03:29:28.250Z,NaN,NaN,2024-10-30T03:20:28.250Z,22086,Charles de Gaulle - Etoile,metro,Paris 17e
4,2024-11-02 22:21:13,463013,Charles de Gaulle-Etoile,STIF:Line::C01371:,metro,La Défense (Grande Arche),delayed,NaN,2024-11-02T21:23:35.241Z,NaN,2024-11-02T21:23:35.241Z,NaN,NaN,2024-11-02T21:20:35.241Z,463013,Charles de Gaulle - Etoile,metro,Paris 17e
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137,2024-11-14 02:20:54,463013,Charles de Gaulle-Etoile,STIF:Line::C01371:,metro,La Défense (Grande Arche),delayed,NaN,2024-11-14T01:47:23.738Z,NaN,2024-11-14T01:47:23.738Z,NaN,NaN,2024-11-14T01:20:23.738Z,463013,Charles de Gaulle - Etoile,metro,Paris 17e
138,2024-11-14 03:20:29,22086,Charles de Gaulle-Etoile,STIF:Line::C01371:,metro,Château de Vincennes,delayed,NaN,2024-11-14T02:14:06.006Z,NaN,2024-11-14T02:14:06.006Z,NaN,NaN,2024-11-14T02:05:06.006Z,22086,Charles de Gaulle - Etoile,metro,Paris 17e
139,2024-11-14 03:21:07,463013,Charles de Gaulle-Etoile,STIF:Line::C01371:,metro,La Défense (Grande Arche),delayed,NaN,2024-11-14T02:32:06.006Z,NaN,2024-11-14T02:32:06.006Z,NaN,NaN,2024-11-14T02:05:06.006Z,463013,Charles de Gaulle - Etoile,metro,Paris 17e
140,2024-11-14 04:20:27,22086,Charles de Gaulle-Etoile,STIF:Line::C01371:,metro,Château de Vincennes,delayed,NaN,2024-11-14T03:28:59.820Z,NaN,2024-11-14T03:28:59.820Z,NaN,NaN,2024-11-14T03:19:59.820Z,22086,Charles de Gaulle - Etoile,metro,Paris 17e


In [48]:
merged_df_rer = pd.merge(delays_r, filtered_data, left_on='stop_reference', right_on='ArRId')
merged_df_rer

,timestamp,stop_reference,stop_name,line_ref,transport_mode,destination_name,departure_status,scheduled_arrival,real_arrival,scheduled_departure,real_departure,arrival_difference,departure_difference,recorded_at_time,ArRId,ArRName,ArRType,ArRTown
0,2024-10-30 05:23:03,41295,Avenue du Président Kennedy,STIF:Line::C01727:,rail,Gare de Pontoise,onTime,2024-10-30T05:05:00.000Z,2024-10-30T05:11:32.000Z,2024-10-30T05:06:00.000Z,2024-10-30T05:12:32.000Z,6.53,6.53,2024-10-30T04:22:24.888Z,41295,Avenue du Président Kennedy,rail,Paris 16e
1,2024-10-30 05:27:09,473921,Châtelet - Les Halles,STIF:Line::C01742:,rail,Saint-Germain-en-Laye,onTime,2024-10-30T04:30:38.000Z,2024-10-30T04:36:50.000Z,2024-10-30T04:30:38.000Z,2024-10-30T04:36:50.000Z,6.20,6.20,2024-10-30T04:12:29.056Z,473921,Châtelet - Les Halles,rail,Paris 1er
2,2024-10-30 06:23:36,41295,Avenue du Président Kennedy,STIF:Line::C01727:,rail,Gare de Pontoise,onTime,2024-10-30T05:05:00.000Z,2024-10-30T05:10:30.000Z,2024-10-30T05:06:00.000Z,2024-10-30T05:11:30.000Z,5.50,5.50,2024-10-30T05:10:34.242Z,41295,Avenue du Président Kennedy,rail,Paris 16e
3,2024-10-30 06:23:36,41295,Avenue du Président Kennedy,STIF:Line::C01727:,rail,Gare de Montigny Beauchamp,onTime,2024-10-30T05:21:20.000Z,2024-10-30T05:33:21.000Z,2024-10-30T05:22:20.000Z,2024-10-30T05:34:21.000Z,12.02,12.02,2024-10-30T05:23:15.385Z,41295,Avenue du Président Kennedy,rail,Paris 16e
4,2024-10-30 07:24:40,41295,Avenue du Président Kennedy,STIF:Line::C01727:,rail,Gare de Montigny Beauchamp,onTime,2024-10-30T06:03:20.000Z,2024-10-30T06:12:30.000Z,2024-10-30T06:04:20.000Z,2024-10-30T06:13:30.000Z,9.17,9.17,2024-10-30T06:12:33.880Z,41295,Avenue du Président Kennedy,rail,Paris 16e
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
768,2024-11-14 09:28:55,473993,Châtelet - Les Halles,STIF:Line::C01742:,rail,Boissy-Saint-Léger,onTime,2024-11-14T07:48:33.000Z,2024-11-14T08:12:53.000Z,2024-11-14T07:48:33.000Z,2024-11-14T08:12:53.000Z,24.33,24.33,2024-11-14T07:48:05.047Z,473993,Châtelet - Les Halles,rail,Paris 1er
769,2024-11-14 09:28:55,473993,Châtelet - Les Halles,STIF:Line::C01742:,rail,Boissy-Saint-Léger,onTime,2024-11-14T08:11:53.000Z,2024-11-14T08:30:08.000Z,2024-11-14T08:11:53.000Z,2024-11-14T08:30:08.000Z,18.25,18.25,2024-11-14T03:54:34.345Z,473993,Châtelet - Les Halles,rail,Paris 1er
770,2024-11-14 09:29:10,474060,Massy - Palaiseau,STIF:Line::C01743:,rail,Saint-Rémy-lès-Chevreuse,onTime,2024-11-14T07:59:19.000Z,2024-11-14T08:09:24.000Z,2024-11-14T07:59:19.000Z,2024-11-14T08:09:24.000Z,10.08,10.08,2024-11-14T07:43:26.800Z,474060,Massy - Palaiseau,rail,Massy
771,2024-11-14 09:29:10,474060,Massy - Palaiseau,STIF:Line::C01743:,rail,Orsay - Ville,onTime,2024-11-14T08:16:52.000Z,2024-11-14T08:25:44.000Z,2024-11-14T08:16:52.000Z,2024-11-14T08:25:44.000Z,8.87,8.87,2024-11-14T07:43:30.667Z,474060,Massy - Palaiseau,rail,Massy


In [ ]:
#drop ArRId, ArRName, ArRType, transport_mode